## 流程
1. 加载数据
2. 数据清洗

In [1]:
import platform
print(platform.python_version())
import datetime
now_time = datetime.datetime.now().strftime('%H:%M, %m-%d, %Y')
print("hello from {} at {}".format(platform.python_version(), now_time))

3.7.11
hello from 3.7.11 at 20:37, 12-21, 2021


In [2]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
SEED = 7
data_path = "G:\\OneDrive - teleworm\\code\\4research\\datasets\\german-credit\\german.csv"
np.random.seed(SEED)
train_df = pd.read_csv(data_path)
train_df.head(5)

,AccountStatus,Duration(months),CreditHistory,Purpose,CreditAmount,SavingsAccount/bonds,PresentEmploymentSince,InstallmentRateRercentage,Marry&Sex,OtherDebtors,...,Property,Age,OtherInstallmentPlan,Housing,ExistingCredits,Job,NumberPeopleLiable,Telephone,foreign,label
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2


In [3]:
#!skelearn imports
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [4]:
target = train_df['Good?']
target[:5]

KeyError: 'Good?'

In [ ]:
#Identify NA values
train_df.isna().sum()


In [ ]:
#!核密度估计
plt.figure(figsize=(20,10))
train_df.InstallmentRateRercentage.plot.kde()

In [ ]:
#!train_df.corr()计算相关系数correlation
print("由下面的相关系数矩阵heatmap可知其中的变量是否存在线性关系")
sns.heatmap(train_df.corr(), vmin=-1, vmax=1,
            annot=True, cmap = "Greens")

In [ ]:
train_df.describe()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_df.iloc[:,:-1], train_df.iloc[:,-1:], test_size = 0.3, random_state = 66)

print(X_train.shape,y_train.shape)
X_train_onehot = pd.get_dummies(X_train)
X_test_onehot  = pd.get_dummies(X_test)
X_train_onehot.head(5)

In [ ]:
#!ExtraTreesClassifier
extra_tree_clf = ExtraTreesClassifier(n_estimators=250, random_state=SEED)
extra_tree_clf.fit(X_train_onehot, y_train)

#!Plot feature importance, make importance relative to max importance
feature_importance = 100.0 * (extra_tree_clf.feature_importances_ / extra_tree_clf.feature_importances_.max())
sorted_idxs = np.argsort(feature_importance)
pos = np.arange(10) + .5
print(sorted_idxs[:10])
print(feature_importance[:10])
print(pos)
#!slice -10, to get top 10 importances
plt.barh(pos[-10:], feature_importance[sorted_idxs][-10:], align='center')
plt.yticks(pos[-10:], X_train_onehot.columns[sorted_idxs][-10:])#boston.feature_names[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

In [ ]:
col_name = list(X_train_onehot.columns)
print(col_name[:5])
#!这样做的目的是为了支持feat_names[indices]的"排序"
feat_names = X_train_onehot[col_name].columns
print(feat_names[:5])

random_forest_classifier = RandomForestClassifier(n_estimators=50,random_state=123)
random_forest_classifier.fit(X_train_onehot, y_train) #对自变量和因变量进行拟合

plt.style.use('ggplot')
## feature importances 可视化##
importances = random_forest_classifier.feature_importances_#记录重要程度

#!将重要性按降序排列[::-1]
indices = np.argsort(importances)[::-1]
fig = plt.figure(figsize=(20,6))
plt.title("Feature importances by RandomTreeClassifier")
#!条形统计图，参数为：条数，每一条的名称，颜色，条形的中心位置
plt.bar(range(len(indices)), importances[indices], color='lightblue',  align="center")
#!阶梯图
plt.step(range(len(indices)), np.cumsum(importances[indices]), where='mid', label='Cumulative')
#!改变坐标轴的名字
plt.xticks(range(len(indices)), feat_names[indices], rotation='vertical',fontsize=14)
#!设置x轴的数值显示范围
plt.xlim([-1, len(indices)])
plt.show()

In [ ]:
#!在测试集合的结果
y_pred = grid_search.predict(X_test_onehot)
from sklearn.metrics import accuracy_score
print("Test set accuracy score: {:.5f}".format(accuracy_score(y_test, y_pred,)))

In [ ]:
#!各指标
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
#构建随机森林进行分类
from sklearn.model_selection import train_test_split
#!使用GridSearchCV进行调参数
from sklearn.model_selection import GridSearchCV
#!构建参数组合, 使用GridSearchCV对'n_estimators'进行调参
param_grid = {'n_estimators': [250,275,300]}
grid_search = GridSearchCV(RandomForestClassifier(),  param_grid, cv=10)
grid_search.fit(X_train_onehot, y_train)

#!查看[训练]最佳结果以及参数
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.5f}".format(grid_search.best_score_))

In [ ]:
#!ROC面积
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_test, y_pred)
print("Area under the ROC curve : %f" % roc_auc)